# Inverse Optimization for DFS
_Applying Ghobadi and Mahmoudzadeh 2021_

In [ ]:
using JuMP
using Gurobi
using LinearAlgebra

## Structures and parameters

In [ ]:
struct ForwardProblemParams
    n_paths::Int
    n_commodities::Int

    capacities::Vector{Number}
    demands::Vector

    design_costs::Vector
    flow_costs::Matrix

    enabled_flows::Matrix{Bool}

    function ForwardProblemParams(; n_paths, n_commodities, capacities, demands, design_costs, flow_costs, enabled_flows=nothing)
        shape = (n_paths, n_commodities)
        
        if isnothing(enabled_flows) 
            enabled_flows = ones(Bool, shape)
        elseif size(enabled_flows) != shape
            error("Invalid shape $(size(enabled_flows)) for `disabled_flows`, should be $(shape)")
        end
        
        new(n_paths, n_commodities, capacities, demands, design_costs, flow_costs, enabled_flows)
    end
end

struct ForwardSolution
    x_sol::Matrix
    z_sol::Vector
end

In [ ]:
function forward_example_params()::ForwardProblemParams
    enabled_flows = ones(Bool, (2, 2))
    enabled_flows[1, 2] = false

    return ForwardProblemParams(
        n_paths=2, 
        n_commodities=2,
        capacities=[100, 100],
        demands=[10, 6],
        design_costs=[100, 10],
        flow_costs=[10 10 ; 100 100],
        enabled_flows=enabled_flows
    )
end

function sol_vector(sol::ForwardSolution)::Vector
    flat_xs = reshape(sol.x_sol, (length(sol.x_sol), 1))

    return vec(vcat(flat_xs, sol.z_sol))
end

## Forward problem

In [ ]:
function create_forward_problem(params::ForwardProblemParams)::Model
    model = Model(Gurobi.Optimizer)

    @variable(model, z[1:params.n_paths], Bin)
    @variable(model, x[1:params.n_paths, 1:params.n_commodities] >= 0)

    @objective(model, Min, params.design_costs' * z + sum(params.flow_costs .* x))

    @constraint(model, [k = 1:params.n_commodities], sum(x[:, k]) .== params.demands[k])
    @constraint(model, [i = 1:params.n_paths], sum(x[i, :]) <= params.capacities[i] * z[i])
    @constraint(model, [i = 1:params.n_paths, k = 1:params.n_paths; !params.enabled_flows[i, k]], x[i, k] .== 0)

    return model
end

function solve_forward_problem!(model::Model)::ForwardSolution
    optimize!(model)

    x_sol = value.(model[:x])
    z_sol = value.(model[:z])

    return ForwardSolution(x_sol, z_sol)
end

In [ ]:
forward_model = create_forward_problem(forward_example_params())
forward_sol = solve_forward_problem!(forward_model)

In [ ]:
println(forward_sol.x_sol)
println(forward_sol.z_sol)

## Inverse problem (demand only)

In [ ]:
struct InverseDemandSolution
    demands::Vector
end

In [ ]:
function create_A_demand(n_paths::Integer, n_commodities::Integer, n_variables::Integer, enabled_flows::Matrix{Bool})
    A_pos = zeros(Number, (n_commodities, n_variables))
    for k in 1:n_commodities
        shift_amount = (k - 1) * n_paths
        shifted_range = (1:n_paths) .+ shift_amount
        A_pos[k, shifted_range] .= Int.(params.enabled_flows[:,k])
    end
    A = vcat(A_pos, -A_pos)

    return A
end

function create_Gh_demand(n_paths::Integer, n_commodities::Integer, n_variables::Integer, enabled_flows::Matrix{Bool})
    n_flows = n_paths * n_commodities

    G_paths = zeros(Number, (n_paths, n_variables))
    for p in 1:n_paths
        G_paths[p, p:n_commodities:n_commodities*n_paths] .= Int.(params.enabled_flows[p, :])
        G_paths[p, n_flows + p] = params.capacities[p]
    end
    G_nonneg = diagm(ones(n_variables))
    G_binary = hcat(zeros((n_paths, n_flows)), diagm(ones(n_paths)))
    
    G = vcat(.-G_paths, G_nonneg, G_binary)
    h = zeros(size(G)[1])

    return G,h
end

function create_AGh_demand(params::ForwardProblemParams)
    n_paths, n_commodities = params.n_paths, params.n_commodities
    n_flows = n_paths * n_commodities
    n_variables = n_flows + n_paths

    A = create_A_demand(n_paths, n_commodities, n_variables, params.enabled_flows)
    G,h = create_Gh_demand(n_paths, n_commodities, n_variables, params.enabled_flows)

    return A, G, h
end
function add_half_space_constraint_demand(G::Matrix, h::Vector, solutions::Tuple{ForwardSolution}, params::ForwardProblemParams)
    flat_flow_costs = reshape(params.flow_costs, (length(params.flow_costs), 1))
    full_costs = vcat(flat_flow_costs, params.design_costs)

    optimal_cost = minimum(sol -> full_costs' * sol_vector(sol), solutions)

    G_hs = vcat(G, full_costs')
    h_hs = vcat(h, optimal_cost)

    return G_hs, h_hs
end

In [ ]:
function create_b_variables_demand!(model::Model, n_commodities::Integer)
    @variable(model, b[1:(2 * n_commodities)])
    @constraint(model, [i = 1:n_commodities], b[i] == -b[n_commodities + i])
    
    return b
end

function add_inverse_constraints_demand!(model::Model, solutions::Tuple{ForwardSolution}, A::Matrix, b::Vector, G::Matrix, h::Vector)
    @constraint(model, [sol in solutions], A*sol_vector(sol) .>= b)
end

function add_inverse_objective_demand!(model::Model, A::Matrix, b::Vector)
end

function create_inverse_demand_problem(params::ForwardProblemParams, solutions::ForwardSolution...)
    model = Model(Gurobi.Optimizer)

    A, G, h = create_AGh_demand(params)
    G, h = add_half_space_constraint_demand(G, h, solutions, params)
    
    b = create_b_variables_demand!(model, params.n_commodities)
    print(typeof(solutions))

    add_inverse_constraints_demand!(model, solutions, A, b, G, h)
    add_inverse_objective_demand!(model, A, b)    

    return model
end

In [ ]:
params = forward_example_params()

forward_model = create_forward_problem(params)
forward_sol = solve_forward_problem!(forward_model)

In [ ]:
inverse_demand_model = create_inverse_demand_problem(params, forward_sol)
print(inverse_demand_model)

In [ ]:
function solve_inverse_demand_problem!(model::Model)::InverseDemandSolution
    optimize!(model)

    b_sol = value.(model[:b])
    half = length(b_sol) ÷ 2
    b_first, b_second = b_sol[1:half], b_sol[half+1:end]
    
    demands = (all(b_first .>= 0)) ? b_first : b_second

    return InverseDemandSolution(demands)
end

In [ ]:
solve_inverse_demand_problem!(inverse_demand_model)

## Inverse problem (demand linear regression)